In [1]:
# Import library and some pre-installed modules
import os
from IPython.display import display, Markdown

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [3]:
# Get current working directory
os.getcwd()


'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

## Import the modules

In [4]:
# Reload the utils and processar_com_overpass modules to ensure any changes are reflected
import importlib
import sys

import src.utils as utils
import src.processar_com_overpass as processar_com_overpass
import src.processar_com_ohsome as processar_com_ohsome

importlib.reload(utils)
importlib.reload(processar_com_overpass)
importlib.reload(processar_com_ohsome)

#Alternativa
#importlib.reload(sys.modules["src.utils"])
#importlib.reload(sys.modules["src.processar_com_overpass"])
#importlib.reload(sys.modules["src.processar_com_ohsome"])

<module 'src.processar_com_ohsome' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/src/processar_com_ohsome.py'>

## Retrieving data from OpenStreetMap using APIs

### Define ET-EDGV class dictionary with respective OSM tags

In [5]:
# Novo dicionário de classes ET-EDGV com respectivas tags OSM
classe_et_edgv_to_tags = {
    'edif_ensino': [
        ('amenity', 'school'), ('amenity', 'university'),
        ('building', 'school'), ('amenity', 'kindergarten')
    ],
    'edif_saude': [
        ('amenity', 'hospital'), ('amenity', 'clinic'),
        ('building', 'hospital'), ('amenity', 'doctors'),
        ('amenity', 'dentist'), ('healthcare', '*')
    ],
    'edif_desenv_social': [
        ('amenity', 'social_facility'), ('building', 'public'),
        ('social_facility', '*')
    ],
    'edif_constr_lazer': [
        ('leisure', 'park'), ('leisure', 'sports_centre'),
        ('leisure', 'stadium'), ('amenity', 'theatre'),
        ('amenity', 'library'), ('amenity', 'community_centre'),
        ('amenity', 'arts_centre'), ('amenity', 'planetarium'),
        ('building', 'grandstand'), ('building', 'stadium'),
        ('tourism', 'museum')
    ],
    'edif_pub_civil': [
        ('building', 'public'), ('amenity', 'townhall'),
        ('office', 'government')
    ],
    'edif_turistica': [
        ('tourism', 'attraction'), ('tourism', 'artwork'),
        ('tourism', 'viewpoint'), ('amenity', 'fountain'),
        ('building', 'hotel')
    ],
    'edif_metro_ferroviaria': [
        ('railway', 'station'), ('railway', 'halt'),
        ('building', 'train_station'), ('public_transport', 'station')
    ]
}
classe_et_edgv_to_tags

{'edif_ensino': [('amenity', 'school'),
  ('amenity', 'university'),
  ('building', 'school'),
  ('amenity', 'kindergarten')],
 'edif_saude': [('amenity', 'hospital'),
  ('amenity', 'clinic'),
  ('building', 'hospital'),
  ('amenity', 'doctors'),
  ('amenity', 'dentist'),
  ('healthcare', '*')],
 'edif_desenv_social': [('amenity', 'social_facility'),
  ('building', 'public'),
  ('social_facility', '*')],
 'edif_constr_lazer': [('leisure', 'park'),
  ('leisure', 'sports_centre'),
  ('leisure', 'stadium'),
  ('amenity', 'theatre'),
  ('amenity', 'library'),
  ('amenity', 'community_centre'),
  ('amenity', 'arts_centre'),
  ('amenity', 'planetarium'),
  ('building', 'grandstand'),
  ('building', 'stadium'),
  ('tourism', 'museum')],
 'edif_pub_civil': [('building', 'public'),
  ('amenity', 'townhall'),
  ('office', 'government')],
 'edif_turistica': [('tourism', 'attraction'),
  ('tourism', 'artwork'),
  ('tourism', 'viewpoint'),
  ('amenity', 'fountain'),
  ('building', 'hotel')],
 'edif

### Step 7 - Retrieval of the last toponyms by the most recent feature

#### PostGIS - Open the database connection

In [10]:
# Conexão ao Banco PostGIS
import psycopg2

# Function to load database credentials from a text file
def load_credentials_from_txt(file_path):
    credentials = {}
    try:
        with open(file_path, 'r') as f:
            for line in f:
                if '=' in line:
                    key, value = line.strip().split('=', 1)
                    credentials[key.strip()] = value.strip()
    except FileNotFoundError:
        print(f"Arquivo de credenciais não encontrado: {file_path}")
    except Exception as e:
        print(f"Erro ao ler credenciais: {e}")
    return credentials

def connect_to_postgis(txt_path='configs/db_credentials.txt'):
    creds = load_credentials_from_txt(txt_path)

    required_keys = ['DB_NAME', 'DB_USER', 'DB_PASSWORD', 'DB_HOST', 'DB_PORT']
    if not all(k in creds for k in required_keys):
        print("Credenciais incompletas no arquivo de configuração.")
        return None

    try:
        conn = psycopg2.connect(
            dbname=creds['DB_NAME'],
            user=creds['DB_USER'],
            password=creds['DB_PASSWORD'],
            host=creds['DB_HOST'],
            port=creds['DB_PORT']
        )
        print("Conexão ao PostGIS estabelecida com sucesso!")
        return conn

    except Exception as e:
        print(f" Erro ao conectar ao PostGIS: {e}")
        return None

In [11]:
# Open the database connection
conn = connect_to_postgis()

Conexão ao PostGIS estabelecida com sucesso!


#### Filter grid cells from the database with name_ratio > 0

 * Pré-filtragem SQL
 * Garantir que pelo menos uma classe tem name_ratio > 0 para cada célula da grade


In [12]:
# Function to fetch valid grid cells from the database
from shapely import wkt
import pandas as pd
import geopandas as gpd

def fetch_geometries_psycopg2(conn, classe_et_edgv_to_tags, table_name):
    """
    Recupera geometrias do PostGIS com base nos filtros name_ratio > 0 usando psycopg2.
    Retorna um GeoDataFrame.
    """
    try:
        # Garante que qualquer erro anterior seja limpo
        conn.rollback()

        # Monta cláusula WHERE
        where_clause = " OR ".join([
            f"step1_consolidado_{classe}_name_ratio > 0" for classe in classe_et_edgv_to_tags
        ])

        query = f"""
            SELECT *, ST_AsText(geom) AS wkt_geom
            FROM public.{table_name}
            WHERE {where_clause}
        """

        with conn.cursor() as cur:
            cur.execute(query)
            colnames = [desc[0] for desc in cur.description]
            rows = cur.fetchall()

        # Monta DataFrame
        df = pd.DataFrame(rows, columns=colnames)

        # Converte geometria WKT em shapely
        df["geometry"] = df["wkt_geom"].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(df.drop(columns=["wkt_geom"]), geometry="geometry")

        # Define CRS padrão
        gdf.set_crs(epsg=4674, inplace=True)

        print(f"Consulta retornou {len(gdf)} registros.")
        return gdf

    except psycopg2.Error as e:
        conn.rollback()
        print("Erro ao executar a consulta SQL:")
        print(e.pgerror)
        raise

In [13]:
# Get the valid grid cell (name_ratio>0)from the database using psycopg2
gdf_cells_valid = fetch_geometries_psycopg2(conn, classe_et_edgv_to_tags, table_name="steps_merged_1to6")
display(gdf_cells_valid.head())

Consulta retornou 2265 registros.


,id,geom,fid,POP10,step1_consolidado_edif_ensino_total_count,step1_consolidado_edif_ensino_name_count,step1_consolidado_edif_ensino_name_ratio,step1_consolidado_edif_saude_total_count,step1_consolidado_edif_saude_name_count,step1_consolidado_edif_saude_name_ratio,...,step6_consolidado_edif_metro_ferroviaria_sigmoid_pct_erro,step6_consolidado_edif_metro_ferroviaria_sigmoid_a,step6_consolidado_edif_metro_ferroviaria_sigmoid_b,step6_consolidado_edif_metro_ferroviaria_sigmoid_c,step6_consolidado_edif_metro_ferroviaria_sigmoid_d,step6_consolidado_edif_metro_ferroviaria_inflexao_idx,step6_consolidado_edif_metro_ferroviaria_inflexao_data,step6_consolidado_edif_metro_ferroviaria_sigmoid_fit_overflow,step6_consolidado_edif_metro_ferroviaria_dias_desde_inflexao,geometry
0,200ME60338N90882,0106000020421200000100000001030000000100000005...,1,345,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((-44.06371 -19.97798, -44.06378..."
1,200ME60346N90862,0106000020421200000100000001030000000100000005...,8,202,1.0,1.0,100.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((-44.05535 -19.99568, -44.05542..."
2,200ME60348N90862,0106000020421200000100000001030000000100000005...,9,525,1.0,1.0,100.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((-44.05343 -19.99561, -44.05350..."
3,200ME60346N90864,0106000020421200000100000001030000000100000005...,11,349,1.0,1.0,100.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((-44.05542 -19.99388, -44.05548..."
4,200ME60348N90864,0106000020421200000100000001030000000100000005...,12,500,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,"MULTIPOLYGON (((-44.05350 -19.99382, -44.05356..."


In [10]:
# Count the number of valid cells (name_ratio > 0) for each class
valid_counts = {
    classe: (gdf_cells_valid[f"step1_consolidado_{classe}_name_ratio"] > 0).sum()
    for classe in classe_et_edgv_to_tags
}
valid_counts

{'edif_ensino': 7,
 'edif_saude': 1,
 'edif_desenv_social': 0,
 'edif_constr_lazer': 34,
 'edif_pub_civil': 1,
 'edif_turistica': 1,
 'edif_metro_ferroviaria': 0}

In [9]:
# Verifica se as colunas estão corretas
gdf_cells_valid.columns.tolist()

['id',
 'POP10',
 'step1_consolidado_edif_ensino_total_count',
 'step1_consolidado_edif_ensino_name_count',
 'step1_consolidado_edif_ensino_name_ratio',
 'step1_consolidado_edif_saude_total_count',
 'step1_consolidado_edif_saude_name_count',
 'step1_consolidado_edif_saude_name_ratio',
 'step1_consolidado_edif_desenv_social_total_count',
 'step1_consolidado_edif_desenv_social_name_count',
 'step1_consolidado_edif_desenv_social_name_ratio',
 'step1_consolidado_edif_constr_lazer_total_count',
 'step1_consolidado_edif_constr_lazer_name_count',
 'step1_consolidado_edif_constr_lazer_name_ratio',
 'step1_consolidado_edif_pub_civil_total_count',
 'step1_consolidado_edif_pub_civil_name_count',
 'step1_consolidado_edif_pub_civil_name_ratio',
 'step1_consolidado_edif_turistica_total_count',
 'step1_consolidado_edif_turistica_name_count',
 'step1_consolidado_edif_turistica_name_ratio',
 'step1_consolidado_edif_metro_ferroviaria_total_count',
 'step1_consolidado_edif_metro_ferroviaria_name_count',


#### **Request last toponyms and metadata – OHSOME API**


1. Conectar ao PostGIS para filtrar apenas células onde name_ratio > 0 para pelo menos uma classe.

2. Pré-filtragem das por células das classe com as tags do dicionário classe_et_edgv_to_tags - aumentar performance.

3. Para cada célula e classe com name_ratio > 0:

  * Extrair bbox da célula válida;

  * Uso do inflexao_data (step6) como início da janela temporal

  * Fazer chamada A API OSHOME para para recuperar a contribuição mais recente com name=*
    - Endpoint: POST /contributions/latest/geometry;

  * ~~'Respeito à data de inflexão (step6_consolidado_{classe}_inflexao_data)~~
    * Essa estratégia deixou de ser utilizada, pois foi observado que a contribuição mais recente com name=* pode ter sido feita antes da data da inflexão.
    * Nesses casos, usando a inflexão como fromTimestamp, ficamos sem qualquer resultado retornado pela API OHSOME

  * Resgatar geometria (pontos) e metadados:
    - Metadados: @timestamp, @osmId, tags, name.

4. Paralelização por célula

5. Log detalhado e consolidação final

6. Salvar os resultados em GeoJSON incremental.

In [7]:
# Fetch metadata from the ohsome API
# This code fetches metadata from the ohsome API and handles potential JSON decoding errors.
import requests

URL = 'https://api.ohsome.org/v1/metadata'
response = requests.get(URL)

if response.status_code == 200:
    try:
        data = response.json()
        print("Dados recebidos:")
        display(data)
    except ValueError:
        print("Erro ao decodificar JSON. Conteúdo bruto:")
        display(response.text)
else:
    display(f"Erro HTTP {response.status_code}")
    print("Resposta:")
    display(response.text)

Dados recebidos:


{'attribution': {'url': 'https://ohsome.org/copyrights',
  'text': '© OpenStreetMap contributors'},
 'apiVersion': '1.10.4',
 'timeout': 600.0,
 'extractRegion': {'spatialExtent': {'type': 'Polygon',
   'coordinates': [[[-180.0, -90.0],
     [180.0, -90.0],
     [180.0, 90.0],
     [-180.0, 90.0],
     [-180.0, -90.0]]]},
  'temporalExtent': {'fromTimestamp': '2007-10-08T00:00:00Z',
   'toTimestamp': '2025-07-01T12:00Z'},
  'replicationSequenceNumber': 112205}}

##### Implementação sem módulos externos

In [ ]:
# === STEP 7: Último topônimo OSM por classe usando a API OHSOME ===
# Versão final com paralelização, logs, output GeoJSON e metadados completos

# Sem módulos externos (Utils), apenas bibliotecas padrão e geopandas

# === Import necessary libraries ===
import requests
import json
from shapely.geometry import shape, mapping
import time
import math
import csv
from pathlib import Path
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import timedelta
import threading
from tqdm import tqdm
import glob

# === CONFIGURAÇÕES GERAIS ===
output_dir = Path("data/output_code1/20cells_tests/step7_latest_name")
output_dir.mkdir(parents=True, exist_ok=True)

log_path = output_dir / "log_step7.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step7.txt"
url_ohsome_latest = "https://api.ohsome.org/v1/contributions/latest/geometry"

# === INICIALIZAÇÃO DO LOG ===
if not log_path.exists():
    with open(log_path, 'w', newline='') as f:
        csv.writer(f).writerow(["lote", "mensagem", "timestamp"])

def log_mensagem(lote, mensagem):
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    with open(log_path, 'a', newline='') as f:
        csv.writer(f).writerow([lote, mensagem, timestamp])

# === KEEP ALIVE (com controle de término) ===
# Flag global para controle
keep_alive_running = True

def keep_alive():
    while keep_alive_running:
        time.sleep(300)
        print("Ainda trabalhando...")
        log_mensagem("keep_alive", "Ainda trabalhando...")

keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

# === FUNÇÃO PARA PROCESSAR UMA CÉLULA ===
def processar_ultima_contribuicao(cell_row):
    id_celula = cell_row["id"]
    bbox = cell_row.geometry.bounds  # (minx, miny, maxx, maxy)
    features_resultantes = []

    for classe, tags in classe_et_edgv_to_tags.items():
        ratio_col = f"step1_consolidado_{classe}_name_ratio"
        inflexao_col = f"step6_consolidado_{classe}_inflexao_data"

        if pd.isna(cell_row.get(ratio_col)) or cell_row[ratio_col] <= 0:
            continue

        data_inicio_str = cell_row.get(inflexao_col) # Uso do inflexao_data como início da janela temporal
        if not isinstance(data_inicio_str, str) or data_inicio_str.strip() == "" or data_inicio_str.lower() == "none":
            continue

        try:
            data_inicio = pd.to_datetime(data_inicio_str, errors="coerce")
            if pd.isna(data_inicio):
                continue
        except Exception as e:
            log_mensagem(id_celula, f"[ERRO] Conversão de data inflexão inválida: {data_inicio_str} - {e}")
            continue

        data_fim = pd.Timestamp("2025-04-06T13:00Z").strftime("%Y-%m-%d") # data_fim fixa de acordo com API metadata ('temporalExtent')
        data_inicio_str = data_inicio.strftime("%Y-%m-%d")

        for tag, value in tags:
            payload = {
                "bboxes": f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
                "time": f"{data_inicio_str},{data_fim}",
                "filter": f"{tag}={value} and name=*",
                "properties": "metadata,tags",
                "clipGeometry": "false"
            }

            try:
                response = requests.post(url_ohsome_latest, data=payload)
                
                print(f"[DEBUG] Célula: {id_celula}, Classe: {classe}, Tag={tag}, Value={value}")
                print(f"[DEBUG] Payload: {json.dumps(payload)}")
                print(f"[DEBUG] Status Code: {response.status_code}")
                print(f"[DEBUG] Response Text: {response.text[:300]}")  # apenas os primeiros 300 chars

                response.raise_for_status()
                dados = response.json()

                for feat in dados.get("features", []):
                    geom_data = feat.get("geometry")
                    props = feat.get("properties", {})

                    if geom_data is None:
                        continue

                    geom = shape(geom_data)
                    if geom.geom_type in ["Polygon", "MultiPolygon"]:
                        geom = geom.centroid

                    props_clean = {
                        "id_celula": id_celula,
                        "classe": classe,
                        "tag": tag,
                        "value": value,
                        **props
                    }

                    features_resultantes.append({
                        "type": "Feature",
                        "geometry": mapping(geom),
                        "properties": props_clean
                    })

            except Exception as e:
                log_mensagem(id_celula, f"[ERRO OHSOME {classe}] {tag}={value}: {str(e)}")

    return features_resultantes

try:
    # === EXECUÇÃO EM LOTE ===
    ultimo_lote = 0
    if ultimo_lote_path.exists():
        with open(ultimo_lote_path, 'r') as f:
            ultimo_lote = int(f.read().strip())

    lote_size = 20
    total_lotes = math.ceil(len(gdf_cells_valid) / lote_size)

    for lote_index in range(ultimo_lote, total_lotes):
        start_time = time.time()
        features_final = []

        subset = gdf_cells_valid.iloc[lote_index * lote_size: (lote_index + 1) * lote_size]

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(processar_ultima_contribuicao, row) for _, row in subset.iterrows()]
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Lote {lote_index + 1} (Step 7)"):
                try:
                    features_final.extend(future.result())
                except Exception as e:
                    log_mensagem(lote_index + 1, f"[FALHA GERAL]: {e}")

        # Salva lote em GeoJSON
        fc = {"type": "FeatureCollection", "features": features_final}
        out_path = output_dir / f"step7_lote{lote_index + 1}.geojson"
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(fc, f)
        log_mensagem(lote_index + 1, f"SALVO {out_path.name}")

        # Atualiza consolidação incremental
        arquivos = sorted(output_dir.glob("step7_lote*.geojson"))
        todas_features = []
        for arquivo in arquivos:
            with open(arquivo, 'r', encoding='utf-8') as f:
                fc_parcial = json.load(f)
                todas_features.extend(fc_parcial['features'])

        final_fc = {"type": "FeatureCollection", "features": todas_features}
        with open(output_dir / "step7_consolidado.geojson", 'w', encoding='utf-8') as f:
            json.dump(final_fc, f)
        log_mensagem(lote_index + 1, "CONSOLIDADO atualizado")

        with open(ultimo_lote_path, 'w') as f:
            f.write(str(lote_index + 1))

        tempo_msg = f"Tempo lote {lote_index + 1}: {str(timedelta(seconds=int(time.time() - start_time)))}"
        print(tempo_msg)
        log_mensagem(lote_index + 1, tempo_msg)

    print("Step 7 (último topônimo por classe) finalizado com sucesso.")
    log_mensagem("step7", "Processamento finalizado")

finally:
    keep_alive_running = False
    keep_alive_thread.join(timeout=1)

##### Implementação com módulos externos

In [ ]:
# === STEP 7 (OHSOME API) ===
# Com módulos utils e processar_com_ohsome

# === Import necessary libraries and modules ===
from src.utils import init_log, start_keep_alive, log_mensagem, consolidar_geojson
from src.processar_com_ohsome import processar_com_ohsome

from tqdm import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import timedelta
import json, math, time

# === CONFIGURAÇÕES GERAIS ===
output_dir = Path("results/2_toponyms_retrieval/step7_latest_name_ohsome/v2_comp/step7_latest_name_ohsome")
output_dir.mkdir(parents=True, exist_ok=True)

log_path = output_dir / "log_step7_ohsome.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step7_ohsome.txt"

# === LOG E KEEP ALIVE ===
# Flag global para controle

init_log(log_path)
keep_alive_flag = {"running": True}
keep_alive_thread = start_keep_alive(log_path, keep_alive_flag)

# === EXECUÇÃO EM LOTE ===
try:
    ultimo_lote = int(ultimo_lote_path.read_text().strip()) if ultimo_lote_path.exists() else 0
    lote_size = 20
    total_lotes = math.ceil(len(gdf_cells_valid) / lote_size)

    for lote_index in range(ultimo_lote, total_lotes):
        start_time = time.time()
        features_final = []
        subset = gdf_cells_valid.iloc[lote_index * lote_size: (lote_index + 1) * lote_size]

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [
                executor.submit(processar_com_ohsome, row, classe_et_edgv_to_tags, log_mensagem, log_path)
                for _, row in subset.iterrows()
            ]
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Lote {lote_index + 1} (Step 7 OHSOME)"):
                try:
                    features_final.extend(future.result())
                except Exception as e:
                    log_mensagem(log_path, lote_index + 1, f"[FALHA GERAL]: {e}")

        out_path = output_dir / f"step7_lote{lote_index + 1}.geojson"
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump({"type": "FeatureCollection", "features": features_final}, f)
        log_mensagem(log_path, lote_index + 1, f"SALVO {out_path.name}")

        total_feats = consolidar_geojson(output_dir, "step7_lote*.geojson", "step7_consolidado_oshome.geojson")
        log_mensagem(log_path, lote_index + 1, f"CONSOLIDADO atualizado: {total_feats} features")

        ultimo_lote_path.write_text(str(lote_index + 1))
        print(f"Lote {lote_index + 1} concluído em {str(timedelta(seconds=int(time.time() - start_time)))}")

    print("Step 7 (OHSOME) finalizado com sucesso.")
    log_mensagem(log_path, "final", "Processamento concluído")

finally:
    keep_alive_flag["running"] = False
    keep_alive_thread.join(timeout=1)

#### **Request last toponyms and metadata – Overpass API**

 * [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API)

##### Implementação sem módulos externos

In [ ]:
# === STEP 7: Último topônimo OSM por classe (via Overpass API) ===
# Sem módulo utils

from pathlib import Path
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
import time, threading, math, json, csv, glob
import pandas as pd

# === CONFIGURAÇÕES GERAIS ===
output_dir = Path("data/output_code1/step7_latest_name_overpass")
output_dir.mkdir(parents=True, exist_ok=True)

log_path = output_dir / "log_step7_overpass.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step7_overpass.txt"

# === INICIALIZAÇÃO DO LOG ===
if not log_path.exists():
    with open(log_path, 'w', newline='') as f:
        csv.writer(f).writerow(["lote", "mensagem", "timestamp"])

def log_mensagem(lote, mensagem):
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    with open(log_path, 'a', newline='') as f:
        csv.writer(f).writerow([lote, mensagem, timestamp])

# === KEEP ALIVE (com controle de término) ===
keep_alive_running = True
def keep_alive():
    while keep_alive_running:
        time.sleep(300)
        print("Ainda trabalhando...")
        log_mensagem("keep_alive", "Ainda trabalhando...")

keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

# === EXECUÇÃO EM LOTE USANDO OVERPASS ===
try:
    ultimo_lote = 0
    if ultimo_lote_path.exists():
        with open(ultimo_lote_path, 'r') as f:
            ultimo_lote = int(f.read().strip())

    lote_size = 20
    total_lotes = math.ceil(len(gdf_cells_valid) / lote_size)

    for lote_index in range(ultimo_lote, total_lotes):
        start_time = time.time()
        features_final = []

        subset = gdf_cells_valid.iloc[lote_index * lote_size: (lote_index + 1) * lote_size]

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(processar_com_overpass, row, classe_et_edgv_to_tags, log_mensagem) for _, row in subset.iterrows()]
            for future in as_completed(futures):
                try:
                    features_final.extend(future.result())
                except Exception as e:
                    log_mensagem(lote_index + 1, f"[FALHA GERAL]: {e}")

        # Salva lote em GeoJSON
        fc = {"type": "FeatureCollection", "features": features_final}
        out_path = output_dir / f"step7_lote{lote_index + 1}.geojson"
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(fc, f)
        log_mensagem(lote_index + 1, f"SALVO {out_path.name}")

        # Atualiza consolidação incremental
        arquivos = sorted(output_dir.glob("step7_lote*.geojson"))
        todas_features = []
        for arquivo in arquivos:
            with open(arquivo, 'r', encoding='utf-8') as f:
                fc_parcial = json.load(f)
                todas_features.extend(fc_parcial['features'])

        final_fc = {"type": "FeatureCollection", "features": todas_features}
        with open(output_dir / "step7_consolidado.geojson", 'w', encoding='utf-8') as f:
            json.dump(final_fc, f)
        log_mensagem(lote_index + 1, "CONSOLIDADO atualizado")

        with open(ultimo_lote_path, 'w') as f:
            f.write(str(lote_index + 1))

        tempo_msg = f"Tempo lote {lote_index + 1}: {str(timedelta(seconds=int(time.time() - start_time)))}"
        print(tempo_msg)
        log_mensagem(lote_index + 1, tempo_msg)

    print("Step 7 (último topônimo por classe - Overpass) finalizado com sucesso.")
    log_mensagem("step7_overpass", "Processamento finalizado")

finally:
    keep_alive_running = False
    keep_alive_thread.join(timeout=1)


##### Implementação com módulos externos

In [ ]:
# === STEP 7 (Overpass API) ===
# Com módulos utils e processar_com_overpass

from src.utils import init_log, start_keep_alive, log_mensagem, consolidar_geojson
from src.processar_com_overpass import processar_com_overpass

from pathlib import Path
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
import math, json, time

# === CONFIGURAÇÃO ===
output_dir = Path("data/output_code1/step7_latest_name_overpass")
output_dir.mkdir(parents=True, exist_ok=True)
log_path = output_dir / "log_step7_overpass.csv"
ultimo_lote_path = output_dir / "ultimo_lote_step7_overpass.txt"

# === LOG E KEEP ALIVE ===
init_log(log_path)
keep_alive_flag = {"running": True}
keep_alive_thread = start_keep_alive(log_path, keep_alive_flag)

# === EXECUÇÃO EM LOTE ===
try:
    ultimo_lote = int(ultimo_lote_path.read_text().strip()) if ultimo_lote_path.exists() else 0
    lote_size = 20
    total_lotes = math.ceil(len(gdf_cells_valid) / lote_size)

    for lote_index in range(ultimo_lote, total_lotes):
        start_time = time.time()
        features_final = []
        subset = gdf_cells_valid.iloc[lote_index * lote_size: (lote_index + 1) * lote_size]

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = [
                executor.submit(processar_com_overpass, row, classe_et_edgv_to_tags, log_mensagem, log_path)
                for _, row in subset.iterrows()
            ]
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"Lote {lote_index + 1} (Step 7 Overpass)"):
                try:
                    features_final.extend(future.result())
                except Exception as e:
                    log_mensagem(log_path, lote_index + 1, f"[FALHA GERAL]: {e}")

        # Salva lote
        out_path = output_dir / f"step7_lote{lote_index + 1}.geojson"
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump({"type": "FeatureCollection", "features": features_final}, f)
        log_mensagem(log_path, lote_index + 1, f"SALVO {out_path.name}")

        # Consolida incremental
        total_feats = consolidar_geojson(output_dir, "step7_lote*.geojson", "step7_consolidado.geojson")
        log_mensagem(log_path, lote_index + 1, f"CONSOLIDADO atualizado: {total_feats} features")

        # Atualiza lote
        ultimo_lote_path.write_text(str(lote_index + 1))
        print(f"Lote {lote_index + 1} concluído em {str(timedelta(seconds=int(time.time() - start_time)))}")

    print("Step 7 (Overpass) finalizado com sucesso.")
    log_mensagem(log_path, "final", "Processamento concluído")

finally:
    keep_alive_flag["running"] = False
    keep_alive_thread.join(timeout=1)

In [ ]:
# === DEBUG MANUAL - PROCESSAR UMA CELULA ===

# Rodar debug manual com apenas uma célula
test_row = gdf_cells_valid.iloc[0]
features = processar_com_overpass.processar_com_overpass(test_row, classe_et_edgv_to_tags, log_mensagem, log_path)

print(json.dumps(features, indent=2))

#### Deduplicação de pontos com topônimos extraídos

In [6]:
# Load again the GeoDataFrame from step 7 the consolidated GeoJSON file
import geopandas as gpd
gdf_step7 = gpd.read_file('results/2_toponyms_retrieval/step7_latest_name_ohsome/step7_consolidado_ohsome.geojson')

In [7]:
display(gdf_step7.head())

,id_celula,classe,tag,value,@changesetId,@contributionChangesetId,@creation,@osmId,@osmType,@timestamp,...,indoor,building:use,training,subject:wikidata,subject:wikipedia,payment:account_cards,shop,int_name,nohousenumber,geometry
0,200ME60346N90864,edif_ensino,amenity,school,56647215,56647215,True,way/172871823,way,2018-02-24 22:13:53+00:00,...,None,None,None,None,None,None,None,None,None,POINT (-44.05395 -19.99415)
1,200ME60338N90882,edif_constr_lazer,leisure,park,105601725,105601725,True,way/511117205,way,2021-05-31 06:56:23+00:00,...,None,None,None,None,None,None,None,None,None,POINT (-44.06307 -19.97691)
2,200ME60348N90864,edif_constr_lazer,leisure,park,12479899,12479899,True,way/172871822,way,2012-07-25 02:06:21+00:00,...,None,None,None,None,None,None,None,None,None,POINT (-44.05274 -19.99391)
3,200ME60344N90866,edif_constr_lazer,leisure,park,12479899,96238688,None,way/172873826,way,2020-12-22 07:32:20+00:00,...,None,None,None,None,None,None,None,None,None,POINT (-44.05562 -19.99122)
4,200ME60346N90862,edif_ensino,amenity,school,56647215,56647215,True,way/172871823,way,2018-02-24 22:13:53+00:00,...,None,None,None,None,None,None,None,None,None,POINT (-44.05395 -19.99415)


In [8]:
# Check the number of features in the GeoDataFrame
print(f"Number of features: {len(gdf_step7)}")

Number of features: 3578


In [14]:
import geopandas as gpd
import pandas as pd

# Corrigir tipos
gdf_step7["@version"] = pd.to_numeric(gdf_step7["@version"], errors='coerce')

# Agrupar por geometrias duplicadas
def selecionar_ponto_preferido(grupo):
    """
    Seleciona o ponto mais apropriado entre duplicatas reais (mesma geometry, timestamp e versão)
    Prioriza o ponto cujo id_celula contém realmente o ponto.
    """
    if len(grupo) == 1:
        return grupo

    # Verifica espacialmente qual ponto realmente está dentro da sua célula
    for idx, row in grupo.iterrows():
        id_celula = row["id_celula"]
        ponto = row["geometry"]
        try:
            poligono = gdf_cells_valid.loc[id_celula, "geometry"]
            if ponto.within(poligono):
                return pd.DataFrame([row])
        except KeyError:
            continue  # id_celula não encontrado

    # Se nenhum "bate" espacialmente, mantém o primeiro
    return grupo.iloc[[0]]

# Aplicar deduplicação por geometria
gdf_filtrado = (
    gdf_step7.sort_values(by=["@timestamp", "@version"], ascending=[False, False])
       .groupby(["geometry", "@timestamp", "@version"], group_keys=False)
       .apply(selecionar_ponto_preferido)
       .reset_index(drop=True)
)
# Reconstrói como GeoDataFrame
gdf_filtrado = gpd.GeoDataFrame(gdf_filtrado, geometry="geometry", crs=gdf_step7.crs)

/var/folders/93/jp7gz1rj0sg95pqlydn2yhk40000gn/T/ipykernel_35682/4031892776.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(selecionar_ponto_preferido)


In [24]:
# Number of features after filtering
print(f"Number of features after filtering: {len(gdf_filtrado)}")

Number of features after filtering: 1626


In [33]:
gdf_filtrado.columns.tolist()

['id_celula',
 'classe',
 'tag',
 'value',
 '@changesetId',
 '@contributionChangesetId',
 '@creation',
 '@osmId',
 '@osmType',
 '@timestamp',
 '@version',
 'addr:housenumber',
 'addr:street',
 'amenity',
 'contact:phone',
 'name',
 'operator',
 'leisure',
 '@geometryChange',
 '@tagChange',
 'addr:city',
 'addr:postcode',
 'addr:suburb',
 'short_name',
 'sport',
 'type',
 'healthcare',
 'healthcare:speciality',
 'operator:type',
 'phone',
 'tourism',
 'opening_hours',
 'landuse',
 'natural',
 'wheelchair',
 'admin_level',
 'source',
 'building',
 'dog',
 'email',
 'fixme',
 'phone_1',
 'alt_name',
 'check_date',
 'brand',
 'brand:wikidata',
 'dispensing',
 'internet_access',
 'internet_access:fee',
 'public_transport',
 'railway',
 'station',
 'subway',
 'wikidata',
 'garden:type',
 'theatre:genre',
 'website',
 'brand:wikipedia',
 'surface',
 'school',
 'emergency',
 'government',
 'office',
 'smoking',
 'building:levels',
 'height',
 'grades',
 'religion',
 'level',
 'drive_through',


In [32]:
gdf_filtrado = gdf_filtrado.drop(columns=["payment:pix"])

In [34]:
# Save the filtered GeoDataFrame to a new GeoPackage file
output_gpkg = 'results/2_toponyms_retrieval/step7_latest_name_ohsome/step7_consolidado_ohsome_filtrado.gpkg'
gdf_filtrado.to_file(output_gpkg, driver='GPKG', encoding='utf-8')

In [16]:
# Save the filtered GeoDataFrame to a new GeoJSON file
output_path = 'results/2_toponyms_retrieval/step7_latest_name_ohsome/step7_consolidado_ohsome_filtrado.geojson'
gdf_filtrado.to_file(output_path, driver='GeoJSON', encoding='utf-8')

#### PostGIS - Close the database connection

In [17]:
# Close the database connection
if conn and conn.closed == 0:
    # conexão ainda aberta
    with conn.cursor() as cur:
        ...
        cur.close()
    conn.close()
    print("Conexão com o banco de dados fechada.")
else:
    print("Conexão com o banco de dados já estava fechada ou não foi estabelecida.")


Conexão com o banco de dados fechada.
